In [1]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['SMB', 'HML', 'Mkt-RF'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = 'F-F_Research_Data_Factors.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [3]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected) / 100
    returns_data = data

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,Mkt-RF,SMB,HML
date,,,
196307,-0.0039,-0.0055,-0.0081
196308,0.0507,-0.0095,0.0165
196309,-0.0157,-0.0030,0.0019
196310,0.0253,-0.0054,-0.0009
196311,-0.0085,-0.0113,0.0171


In [4]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [5]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data) - 59):
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + 59]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
    
    # udregner profit, volatilitet og sharpe
    tl = np.dot(np.linalg.inv(cov_matrix), np.ones(3))
    nl = np.dot(np.ones(3), np.dot(np.linalg.inv(cov_matrix), np.ones(3)))
    weights = tl / nl
    
    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + 60:month + 61].mean())
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = returns / volatility

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [6]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [7]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [8]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,Mkt-RF Weight,SMB Weight,HML Weight
0,0.030255,0.014479,2.089524,0.107860,0.231571,0.660569
1,0.013485,0.014389,0.937169,0.106999,0.232948,0.660053
2,0.013528,0.014664,0.922489,0.137697,0.234478,0.627825
3,0.017594,0.014667,1.199595,0.140273,0.233366,0.626361
4,0.007144,0.014688,0.486408,0.132848,0.237271,0.629882


In [9]:
df['Sharpe Ratio'].mean()

0.21371972816610146

In [10]:
len(df)

438